In [2]:
pip install opencv-python --user

  Using cached opencv_python-4.5.4.60-cp39-cp39-win_amd64.whl (35.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2

In [12]:
cap = cv2.VideoCapture(0)

In [3]:
while True:
    success, img = cap.read()
    
    
    img = cv2.flip(img, 0)
    cv2.imshow("OpenBoard", img)
    
    
    if cv2.waitKey(1) == 27:
        break
        
        
cap.release()
cv2.destroyAllWindows()

In [7]:
cap.release()
cv2.destroyAllWindows()

In [5]:
print(img.shape)

(480, 640, 3)


In [2]:
ml = 150
max_x = ml + 250
max_y = 50

In [3]:
tool = cv2.imread("tool.png")
tool = tool.astype('uint8')

In [22]:
cap = cv2.VideoCapture(0)
while True:
    success, img = cap.read()
    
    
    img = cv2.flip(img, 1)
    
    img[:max_y, ml:max_x] = cv2.addWeighted(tool, 1, img[:max_y, ml:max_x], 0, 1)
    cv2.imshow("OpenBoard", img)
    

    if cv2.waitKey(1) == 27:
        break
        
        
cap.release()
cv2.destroyAllWindows()

In [4]:
import mediapipe as mp

In [5]:
hands = mp.solutions.hands
hand_lankmark = hands.Hands(min_detection_confidence = 0.6, min_tracking_confidence = 0.6, max_num_hands = 1)
draw = mp.solutions.drawing_utils

In [6]:
def getTool(x):
    if(x < 50 + ml):
        return "line"
    elif x < 100 + ml:
        return "rectangle"
    
    elif x < 150 + ml:
        return "draw"
    
    elif x < 200 + ml:
        return "circle"
    
    else:
        return "eraser"

In [8]:
def index_raised(yi, y9):
    if (y9 - yi) > 40:
        return True
    else:
        return False

In [12]:
prev_x = 0
prev_y = 0
t = 4

import numpy as np

In [22]:
flag = False

In [ ]:
cap = cv2.VideoCapture(0)
curr_tool = "Select Tool"

mask = np.ones((480, 640)) * 255
mask = mask.astype("uint8")

while True:
    success, img = cap.read()
    
    
    img = cv2.flip(img, 1)
    
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    result = hand_lankmark.process(rgb)
    
    if result.multi_hand_landmarks:
        for i in result.multi_hand_landmarks:
            draw.draw_landmarks(img, i, hands.HAND_CONNECTIONS)
            
            x, y = int(i.landmark[8].x * 640), int(i.landmark[8].y * 480)
            
            if x < max_x and y < max_y and x > ml:
                curr_tool = getTool(x)
#                 print(curr_tool)
                
            if curr_tool == "draw":
                    xi, yi = int(i.landmark[12].x * 640), int(i.landmark[12].y * 480)
                    y9 = int(i.landmark[9].y * 480)
                    
                    if index_raised(yi, y9):
                        cv2.line(mask, (prev_x, prev_y), (x, y), 0, t)
                        
                        prev_x = x
                        prev_y = y
            elif curr_tool == "line":
                xi, yi = int(i.landmark[12].x * 640), int(i.landmark[12].y * 480)
                y9 = int(i.landmark[9].y * 480)

                if index_raised(yi, y9):
                    if not(flag):
                        xii, yii = x, y
                        flag = True

                    cv2.line(img, (xii, yii), (x, y), (136, 46, 167), t)

                else:
                    if flag:
                        cv2.line(mask, (xii, yii), (x, y), 0, t)
                        flag = False
                        
                        
            elif curr_tool == "rectangle":
                xi, yi = int(i.landmark[12].x * 640), int(i.landmark[12].y * 480)
                y9 = int(i.landmark[9].y * 480)

                if index_raised(yi, y9):
                    if not(flag):
                        xii, yii = x, y
                        flag = True

#                     cv2.rectangle(img, (xii, yii), (x, y), (136, 46, 167), t)
                    cv2.circle(img,(x,y),(((x-xii) ** 2 + (y-yii))^2) ** 0.5,(255,255,255),6)



                else:
                    if flag:
                        cv2.rectangle(mask, (xii, yii), (x, y), 0, t)
                        flag = False
                        
            elif curr_tool == "eraser":
                xi, yi = int(i.landmark[12].x * 640), int(i.landmark[12].y * 480)
                y9 = int(i.landmark[9].y * 480)

                if index_raised(yi, y9):
                    cv2.circle(img, (x, y), 50, (135, 56, 234), -1)
                    cv2.circle(mask, (x, y), 50, (255, 255, 255), -1)
                
                
    
    img = cv2.bitwise_and(img, img, mask = mask)
            
    cv2.putText(img, curr_tool, (270+ml, 30), cv2.FONT_HERSHEY_SIMPLEX, 1 ,(0, 0, 255),2)
    img[:max_y, ml:max_x] = cv2.addWeighted(tool, 1, img[:max_y, ml:max_x], 0, 1)
    cv2.imshow("OpenBoard", img)
    

    if cv2.waitKey(1) == 27:
        break
        
        
cap.release()
cv2.destroyAllWindows()